# HER2 One Scanner - Aperio FDA

- 5-Fold (80/20) split, No Holdout Set
- Truth = Categorical from Mean of 7 continuous scores 
- Epoch at automatic Stop when loss<.001 change 
- LeNet model, 10 layers, Dropout (0.7)

In [1]:
import numpy as np
import pandas as pd
import random
from keras.callbacks import EarlyStopping
from PIL import Image
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, Lambda
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.cross_validation import train_test_split
from sklearn.metrics import roc_curve, auc, classification_report
import csv
import cv2
import scipy
import os
%matplotlib inline
import matplotlib.pyplot as plt

Using TensorFlow backend.
/usr/lib/python2.7/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
#For single scanner
BASE_PATH = '/home/diam/Desktop/HER2_data_categorical/Aperio_FDA/'
#BASE PATH for working from home:
#BASE_PATH = '/home/OSEL/Desktop/HER2_data_categorical/'
#epochs = 10
batch_size = 32
num_classes = 3
#epochs = 35

## Get Data - Practice

In [3]:
#This is the version from Ravi's code:

#FDA
#X_FDA = []
#idx_FDA = []
#for index, image_filename in list(enumerate(BASE_PATH)):
#	img_file = cv2.imread(BASE_PATH + '/' + image_filename)
#	if img_file is not None:
		#img_file = smisc.imresize(arr = img_file, size = (600,760,3))
#		img_file = smisc.imresize(arr = img_file, size = (120,160,3))		
#		img_arr = np.asarray(img_file)
#		X_FDA.append(img_arr)
#		idx_FDA.append(index)

#X_FDA = np.asarray(X_FDA)
#idx_FDA = np.asarray(idx_FDA)
#random.seed(rs)
#random_id = random.sample(idx_FDA, len(idx_FDA)/2)
#random_FDA = []
#for i in random_id:
#	random_FDA.append(X_FDA[i])

#random_FDA = np.asarray(random_FDA)

## Get Data - Real

In [4]:
def get_data(folder):
    X = []
    y = []
    filenames = []

    for hclass in os.listdir(folder):
        if not hclass.startswith('.'):
            if hclass in ["0"]:
                label = 0
            else: #label must be 1 or 2
                if hclass in ["1"]:
                    label = 1
                else:
                    label = 2
            for image_filename in os.listdir(folder + hclass):
                filename = folder + hclass + '/' + image_filename
                img_file = cv2.imread(folder + hclass + '/' + image_filename)
                
                if img_file is not None:
                    img_file = scipy.misc.imresize(arr=img_file, size=(120, 160, 3))
                    img_arr = np.asarray(img_file)
                    X.append(img_arr)
                    y.append(label)
                    filenames.append(filename)

    X = np.asarray(X)
    y = np.asarray(y)
    z = np.asarray(filenames)
    return X,y,filenames

In [5]:
X, y, z = get_data(BASE_PATH)


#print(X)
#print(y)
#print(z)
print(len(X))
print(len(y))
print(len(z))

#INTEGER ENCODE
#https://machinelearningmastery.com/how-to-one-hot-encode-sequence-data-in-python/
encoder = LabelEncoder()
y_cat = np_utils.to_categorical(encoder.fit_transform(y))
print(y_cat)





/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:20: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


241
241
241
[[ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.

### Old Code

In [6]:

#encoder = LabelEncoder()
#encoder.fit(y)

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

#encoded_y_train = encoder.transform(y_train)
#encoded_y_test = encoder.transform(y_test)

#y_train = np_utils.to_categorical(encoded_y_train)
#y_test = np_utils.to_categorical(encoded_y_test)

In [7]:
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

## Fit Model with K-Fold X-Val

In [8]:
kf = KFold(n_splits = 5, random_state=5, shuffle=True)
print(kf.get_n_splits(y))
print(kf)


#for train_index, test_index in kf.split(y):
#    X_train, X_test = X[train_index], X[test_index]
#    print(train_index, test_index)




5
KFold(n_splits=5, random_state=5, shuffle=True)


In [9]:

oos_y = []
oos_pred = []
fold = 0


for train, test in kf.split(y_cat):
    fold+=1
    print("fold #{}".format(fold))
        
    X_train = X[train]
    y_train = y_cat[train]
    X_test = X[test]
    y_test = y_cat[test]
    
    #encoder = LabelEncoder()
    #encoder.fit(y_test)
    #y_train = np_utils.to_categorical(encoder.transform(y_train))
    #y_test = np_utils.to_categorical(encoder.transform(y_test))
    
    model = Sequential()
    model.add(Lambda(lambda x: x * 1./255., input_shape=(120, 160, 3), output_shape=(120, 160, 3)))
    model.add(Conv2D(32, (3, 3), input_shape=(120, 160, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
    model.add(Dense(64))
    model.add(Activation('relu'))
    model.add(Dropout(0.7))
    model.add(Dense(3))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy',
                optimizer='rmsprop',
                metrics=['accuracy'])
    
    monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=25, verbose=1, mode='auto')
    
    model.fit(
            X,
            y_cat,
            validation_data=(X_test,y_test),
            callbacks=[monitor],
            shuffle=True,
            batch_size=batch_size,
            verbose=1,
            epochs=1000)
        
    pred = model.predict(X_test)
        
    oos_y.append(y_test)
    pred = np.argmax(pred,axis=1)
    oos_pred.append(pred)
    
        #measure the fold's accuracy
    y_compare = np.argmax(y_test,axis=1) #for accuracy calculation
    score = metrics.accuracy_score(y_compare, pred)
    print("Fold Score (accuracy): {}".format(score))
    print(pred)

fold #1
Train on 241 samples, validate on 49 samples
Epoch 1/1000
241/241 [==============================] - 2s 9ms/step - loss: 1.2005 - acc: 0.5187 - val_loss: 1.0059 - val_acc: 0.4694
Epoch 2/1000
241/241 [==============================] - 0s 1ms/step - loss: 1.0240 - acc: 0.5477 - val_loss: 0.9548 - val_acc: 0.4082
Epoch 3/1000
241/241 [==============================] - 0s 1ms/step - loss: 1.0413 - acc: 0.5062 - val_loss: 1.0083 - val_acc: 0.5306
Epoch 4/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.8806 - acc: 0.5726 - val_loss: 0.9049 - val_acc: 0.5714
Epoch 5/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.7851 - acc: 0.6680 - val_loss: 0.6530 - val_acc: 0.8163
Epoch 6/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.8278 - acc: 0.6680 - val_loss: 0.7048 - val_acc: 0.6735
Epoch 7/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.8295 - acc: 0.6598 - val_loss: 0.9497 - val_acc: 0.5714
Epoch 8/1

241/241 [==============================] - 0s 1ms/step - loss: 0.2850 - acc: 0.8921 - val_loss: 0.2614 - val_acc: 0.8571
Epoch 62/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.2494 - acc: 0.8838 - val_loss: 0.2343 - val_acc: 0.8571
Epoch 63/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.2841 - acc: 0.8672 - val_loss: 0.3282 - val_acc: 0.8571
Epoch 64/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.3297 - acc: 0.8631 - val_loss: 0.3073 - val_acc: 0.8571
Epoch 65/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.2553 - acc: 0.8797 - val_loss: 0.2098 - val_acc: 0.8571
Epoch 66/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.3988 - acc: 0.8631 - val_loss: 0.1900 - val_acc: 0.8776
Epoch 67/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.2348 - acc: 0.8921 - val_loss: 0.3464 - val_acc: 0.7959
Epoch 68/1000
241/241 [==============================] - 0s 1ms/step 

241/241 [==============================] - 0s 1ms/step - loss: 0.1270 - acc: 0.9461 - val_loss: 0.0452 - val_acc: 1.0000
Epoch 122/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.0681 - acc: 0.9876 - val_loss: 0.0339 - val_acc: 1.0000
Epoch 123/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.1345 - acc: 0.9378 - val_loss: 0.0422 - val_acc: 1.0000
Epoch 124/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.5164 - acc: 0.8382 - val_loss: 0.0978 - val_acc: 0.9796
Epoch 125/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.1247 - acc: 0.9461 - val_loss: 0.0558 - val_acc: 1.0000
Epoch 126/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.0893 - acc: 0.9710 - val_loss: 0.0430 - val_acc: 1.0000
Epoch 127/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.0741 - acc: 0.9751 - val_loss: 0.0330 - val_acc: 1.0000
Epoch 128/1000
241/241 [==============================] - 0s 1m

241/241 [==============================] - 0s 1ms/step - loss: 0.0825 - acc: 0.9627 - val_loss: 0.0078 - val_acc: 1.0000
Epoch 182/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.0297 - acc: 0.9917 - val_loss: 0.1046 - val_acc: 0.9388
Epoch 183/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.0938 - acc: 0.9627 - val_loss: 0.0021 - val_acc: 1.0000
Epoch 184/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.0693 - acc: 0.9627 - val_loss: 0.0016 - val_acc: 1.0000
Epoch 185/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.0666 - acc: 0.9793 - val_loss: 0.0038 - val_acc: 1.0000
Epoch 186/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.0492 - acc: 0.9834 - val_loss: 9.7847e-04 - val_acc: 1.0000
Epoch 187/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.3322 - acc: 0.9461 - val_loss: 0.8660 - val_acc: 0.7959
Epoch 188/1000
241/241 [==============================] - 0

241/241 [==============================] - 0s 1ms/step - loss: 0.3535 - acc: 0.8755 - val_loss: 0.9373 - val_acc: 0.7500
Epoch 48/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.4463 - acc: 0.8672 - val_loss: 0.2846 - val_acc: 0.8542
Epoch 49/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.2997 - acc: 0.8797 - val_loss: 0.7948 - val_acc: 0.7083
Epoch 50/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.5540 - acc: 0.8299 - val_loss: 0.2703 - val_acc: 0.8542
Epoch 51/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.2505 - acc: 0.8921 - val_loss: 0.2315 - val_acc: 0.8750
Epoch 52/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.3260 - acc: 0.8755 - val_loss: 0.8027 - val_acc: 0.7500
Epoch 53/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.4275 - acc: 0.8672 - val_loss: 0.2745 - val_acc: 0.8542
Epoch 54/1000
241/241 [==============================] - 0s 1ms/step 

...

Epoch 96/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.2621 - acc: 0.8963 - val_loss: 0.1925 - val_acc: 0.9167
Epoch 97/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.3977 - acc: 0.8548 - val_loss: 0.1695 - val_acc: 0.9167
Epoch 98/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.2152 - acc: 0.8921 - val_loss: 0.1491 - val_acc: 0.9167
Epoch 99/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.2472 - acc: 0.9046 - val_loss: 0.1526 - val_acc: 0.9167
Epoch 100/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.3436 - acc: 0.8714 - val_loss: 0.4129 - val_acc: 0.8333
Epoch 101/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.3175 - acc: 0.8631 - val_loss: 0.1643 - val_acc: 0.9167
Epoch 102/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.2313 - acc: 0.8921 - val_loss: 0.1408 - val_acc: 0.9167
Epoch 103/1000
241/241 [============================

241/241 [==============================] - 0s 1ms/step - loss: 0.1034 - acc: 0.9419 - val_loss: 0.0355 - val_acc: 1.0000
Epoch 157/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.0789 - acc: 0.9668 - val_loss: 0.0260 - val_acc: 1.0000
Epoch 158/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.7680 - acc: 0.8257 - val_loss: 0.1246 - val_acc: 1.0000
Epoch 159/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.1327 - acc: 0.9336 - val_loss: 0.0473 - val_acc: 1.0000
Epoch 160/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.1214 - acc: 0.9585 - val_loss: 0.0385 - val_acc: 1.0000
Epoch 161/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.0844 - acc: 0.9585 - val_loss: 0.0309 - val_acc: 1.0000
Epoch 162/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.1400 - acc: 0.9336 - val_loss: 1.1740 - val_acc: 0.7083
Epoch 163/1000
241/241 [==============================] - 0s 1m

241/241 [==============================] - 0s 2ms/step - loss: 0.0655 - acc: 0.9751 - val_loss: 0.0124 - val_acc: 1.0000
Epoch 217/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.7199 - acc: 0.8838 - val_loss: 0.0129 - val_acc: 1.0000
Epoch 218/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.0590 - acc: 0.9793 - val_loss: 0.0096 - val_acc: 1.0000
Epoch 219/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.0727 - acc: 0.9627 - val_loss: 0.0098 - val_acc: 1.0000
Epoch 220/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.0444 - acc: 0.9793 - val_loss: 0.0059 - val_acc: 1.0000
Epoch 00220: early stopping
Fold Score (accuracy): 1.0
[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1]
fold #5
Train on 241 samples, validate on 48 samples
Epoch 1/1000
241/241 [==============================] - 1s 3ms/step - loss: 1.0937 - acc: 0.4772 - val_loss: 0.9977 - val_acc: 0

241/241 [==============================] - 0s 1ms/step - loss: 0.2254 - acc: 0.8963 - val_loss: 0.4148 - val_acc: 0.9167
Epoch 56/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.3792 - acc: 0.8755 - val_loss: 0.1384 - val_acc: 0.9375
Epoch 57/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.2972 - acc: 0.8838 - val_loss: 0.1879 - val_acc: 0.8958
Epoch 58/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.2172 - acc: 0.9046 - val_loss: 0.1777 - val_acc: 0.9375
Epoch 59/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.2403 - acc: 0.9087 - val_loss: 0.0945 - val_acc: 0.9375
Epoch 60/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.6517 - acc: 0.8216 - val_loss: 0.1542 - val_acc: 0.9375
Epoch 61/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.2110 - acc: 0.9046 - val_loss: 0.1066 - val_acc: 0.9375
Epoch 62/1000
241/241 [==============================] - 0s 1ms/step 

241/241 [==============================] - 0s 2ms/step - loss: 0.0508 - acc: 0.9834 - val_loss: 0.0129 - val_acc: 1.0000
Epoch 116/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.0753 - acc: 0.9751 - val_loss: 0.0137 - val_acc: 1.0000
Epoch 117/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.2917 - acc: 0.9378 - val_loss: 1.9606 - val_acc: 0.6667
Epoch 118/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.3557 - acc: 0.9253 - val_loss: 0.0557 - val_acc: 0.9583
Epoch 119/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.1177 - acc: 0.9378 - val_loss: 0.0191 - val_acc: 1.0000
Epoch 120/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.0682 - acc: 0.9751 - val_loss: 0.0703 - val_acc: 0.9792
Epoch 121/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.0742 - acc: 0.9751 - val_loss: 0.0125 - val_acc: 1.0000
Epoch 122/1000
241/241 [==============================] - 0s 1m